# Skin diseases classfication using image processing and machine learnings technqiues

## Import required libraries

In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from skimage import color
from skimage.feature import hog
from skimage import data, exposure
import numpy as np
import mahotas
import cv2
import os
import h5py

/home/rassh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Training Path

In [5]:
# Training data path
train_path = "dataset/train"

# Fixes image size
fixed_size = tuple((250, 250))


## Feature extacted using Histogram of oreinted gradiented 

In [6]:


def hog_ext(image):
    img = color.rgb2gray(image)
    fd, hog_image = hog(img, orientations=7, 
                        pixels_per_cell=(16, 16),cells_per_block=(1, 1), 
                        block_norm='L1', feature_vector=True, visualise=True)
    hog_img=hog_image.reshape(-1)
    return hog_img

In [7]:
# obtaining training labels
train_labels = os.listdir(train_path)

# Train labels sorted
train_labels.sort()
print(train_labels)

# Empty lists is used to hold feature vectors and labels
hog_features = []
labels = []
i, j = 0, 0
k = 0

# Total number of images per class
num_of_images = 107

['TinaCorporis', 'TineaCruris', 'TineaFaceii']


## Feature Extraction

In [9]:
# loop over folders to exract features from each image in it.

for training_name in train_labels:
    # join the training data path and each species training folder
    dir = os.path.join(train_path, training_name)
    print(training_name)

    # get the current training label
    current_label = training_name

    k = 1
    # loop over the images in each sub-folder
    for x in range(1,108):
        # get the image file name
        file = dir + "/" + str(x) + ".JPG"

        # read the image and resize it to a fixed-size
        image = cv2.imread(file)
        image = cv2.resize(image, fixed_size)
        
        # HOG function is call to extract each feature image
        res1 = hog_ext(image)
        
        # concating features
        hog_feature = np.hstack([res1])

        # update the list of labels and feature vectors
        labels.append(current_label)
        hog_features.append(hog_feature)

        i += 1
        k += 1
    print ("Processed folder: {}".format(current_label))
    j += 1

print ("Feature Extracted")

TinaCorporis


/home/rassh/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


Processed folder: TinaCorporis
TineaCruris
Processed folder: TineaCruris
TineaFaceii
Processed folder: TineaFaceii
Feature Extracted


In [11]:
# get the overall feature vector size
print ("Feature vector size {}".format(np.array(hog_features).shape))

# get the overall training label size
print ("Training Labels {}".format(np.array(labels).shape))

# encode the target labels
targetNames = np.unique(labels)
le = LabelEncoder()
target = le.fit_transform(labels)
print ("Training labels encoded...")

Feature vector size (321, 62500)
Training Labels (321,)
Training labels encoded...


In [12]:
# normalize the feature vector in the range (0-1)
scaler = MinMaxScaler(feature_range=(0, 1))
rescaled_features = scaler.fit_transform(hog_features)
print ("[STATUS] feature vector normalized...")

print ("[STATUS] target labels: {}".format(target))
print ("[STATUS] target labels shape: {}".format(target.shape))

[STATUS] feature vector normalized...
[STATUS] target labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[STATUS] target labels shape: (321,)


In [15]:
# save the feature vector using HDF5
h5f_data = h5py.File('output/data.h5', 'w')
h5f_data.create_dataset('dataset_1', data=np.array(rescaled_features))

h5f_label = h5py.File('output/labels.h5', 'w')
h5f_label.create_dataset('dataset_1', data=np.array(target))

h5f_data.close()
h5f_label.close()

print ("TRAINING ENDED")

TRAINING ENDED
